# Thermal IR Tutorial 0: Downloading datasets for this tutorial

Download through NASA EarthData API https://nsidc.org/data/SNEX20_VPTS_Raw/versions/1

**At the conclusion of this tutorial, you will be able to:**
- Download datasets through the NASA Earthdata API

In [1]:
# Import the packages we'll need
import pandas as pd
from earthdata_api import earthdata_granule_search # for interfacing with the NASA Earthdata API

I'm going to use this `earthdata_granule_search` function which can return a list of URLs that we can then use a utility like wget (with our EarthData credentials) to download the files. Alternatively, we could search on the Earthdata website itself and download through the web interface (which is quite a good tool in my opinion).

---

First, set up some of the search parameters I'll use for all the datasets I want to find: a bounding box for the Grand Mesa area, and a date/time that I want to look for. 

Define a bounding box at the study area I want to look at. Earthdata will return anything that overlaps this spot.
What I'm specifying is a a tiny area, basicallcy a point search, but we can expand to a larger area if needed.

Format like `bounding_box = [ lower left longitude, lower left latitude, upper right longitude, upper right latitude ]`

In [2]:
bounding_box = [-108.19, 39.03, -108.18, 39.04] # a little box on top of Grand Mesa

I am going to look at one specifc day, but we could search multiple dates/times that don't have to be a continous time period by providing a list of date/times (that was my original motivation in writing this function). Use either pandas.Timestamp, datetime.datetime, or numpy.datetime64 objects.

In [3]:
timestampsUTC = pd.Timestamp('2020-2-08') # February 8, 2020

---
Now we can download the first dataset we want, ASTER satellite images. Specificallfy we are going to download the AST_L1T product. Looking its product documentation we see that it is available either as an HDF file with all bands, or as two tif files, one with visible bands and the other with the thermal bands. 

We'll use the geotiff files today.

In [4]:
# Product information for the ASTER products I want
product="AST_L1T" # product name
version="003" # product version
ext="tif" # geotiff files only

To make sure we find the images from this day, we need to specify a window in time to search, which starts at our "timestampsUTC" time.

To define this time window we specify two things, the size of the time window from our start time, and units of that size value.

In [5]:
search_time_window=1 # specify that we want to have a time window of 1
search_time_window_units="d" # specify time window units of Days, which together gives us 1 day

We also want to save the list of data access URLs so we can use wget to download them

In [6]:
list_filepath="data/aster_download_list.txt"

Finally, make the API call to get the list of files that match our search

In [7]:
url_list = earthdata_granule_search(product, version, 
                                    bounding_box, timestampsUTC, 
                                    search_time_window, search_time_window_units, 
                                    list_filepath, ext)
# we can print the list of files, but these are also saved to a file
print(url_list)

Only including tif files in list
Writing list of URLs to data/aster_download_list.txt
['https://e4ftl01.cr.usgs.gov//ASTER_L1T/ASTT/AST_L1T.003/2020.02.08/AST_L1T_00302082020180748_20200209065849_17218_T.tif', 'https://e4ftl01.cr.usgs.gov//ASTER_L1T/ASTT/AST_L1T.003/2020.02.08/AST_L1T_00302082020180748_20200209065849_17218_V.tif']


Now we can open a terminal and use wget to download the files that are listed.

Open a terminal and navigate to the data directory for this tutorial

`cd data`

Use wget to download the files listed in aster_download_list.txt. This requires that you put your Earthdata username wherer it says YOUR_USERNAME. This will also prompt you for your Earthdata password.

`wget --http-user=YOUR_USERNAME --ask-password --keep-session-cookies --auth-no-challenge=on -c -i aster_download_list.txt`

You should now see the AST_L1T geotiff files in your data directory!

---
Now download the airborne TIR imagery for the same day

In [8]:
#product = ""
#version = ""
#list_filepath="data/airborne_ir_download_list.txt"

In [9]:
#url_list = earthdata_granule_search(product, version, 
#                                    bounding_box, timestampsUTC, 
#                                    search_time_window, search_time_window_units, 
#                                    list_filepath)
#print(url_list)

---
Finally download the ground-based temperature measruements for one site

https://nsidc.org/data/SNEX20_VPTS_Raw/versions/1

In [10]:
product = "SNEX20_VPTS_Raw"
version = "1"
list_filepath="data/snow_temperature_download_list.txt"

In [11]:
url_list = earthdata_granule_search(product, version, 
                                    bounding_box, timestampsUTC, 
                                    search_time_window, search_time_window_units, 
                                    list_filepath)
print(url_list)

Writing list of URLs to data/snow_temperature_download_list.txt
['https://n5eil01u.ecs.nsidc.org/DP6/SNOWEX/SNEX20_VPTS_Raw.001/2020.02.05/SNEX20_VPTS_Raw.zip']


---

Use wget to download the airborne IR imagery and the ground-based snow temperatue data